## Difficult estimation

In [1]:
import numpy as np
import random
from pysat.solvers import Solver
from collections import deque
from scipy.ndimage import label
from scipy.spatial import distance
import matplotlib.pyplot as plt
import itertools 
import z3
import json

In [ ]:
dataset = np.load("star_battle_dataset.npz")
puzzles = dataset["puzzles"]
solutions = dataset["solutions"]

print(len(puzzles))

In [5]:
# Show 5 puzzles
for i in range(len(puzzles)):
    print(f"Showing puzzle {i+1}/{len(puzzles)}")
    print(puzzles[i])
    print(solutions[i])

Showing puzzle 1/1
[[1 1 1 1 8 8 8 8 8 3]
 [1 1 1 1 6 6 6 6 3 3]
 [1 1 1 2 6 6 6 3 3 3]
 [1 1 2 2 2 6 0 3 3 3]
 [1 5 2 2 2 6 0 3 3 3]
 [5 5 5 2 2 4 0 0 0 0]
 [5 5 5 5 4 4 4 4 4 9]
 [5 5 5 5 4 4 4 9 9 9]
 [5 5 5 5 9 9 9 9 9 9]
 [5 7 7 7 7 7 7 9 9 9]]
[[0 0 0 0 1 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 0]
 [0 1 0 1 0 0 0 0 0 0]]


In [6]:
#hand written heuristic solving


def zero_surrounding_cells(answer):
    # Get the dimensions of the answer grid
    rows, cols = answer.shape
    
    # Iterate through each cell in the grid
    for row in range(rows):
        for col in range(cols):
            # If there's a star (1) at this position
            if answer[row, col] == 1:
                # Set the surrounding cells to 0 (orthogonally and diagonally)
                for d_row in [-1, 0, 1]:
                    for d_col in [-1, 0, 1]:
                        # Skip the star itself (the current cell)
                        if d_row == 0 and d_col == 0:
                            continue
                        
                        # Calculate the neighboring cell's coordinates
                        new_row, new_col = row + d_row, col + d_col
                        
                        # Check if the neighbor is within bounds
                        if 0 <= new_row < rows and 0 <= new_col < cols:
                            answer[new_row, new_col] = 0

    return answer


#function to update the puzzle with the stars with no cell "removed"]
def mark_no_star_cells(answer):
    # Find all cells that are definitely no stars
    no_star_cells = np.argwhere(answer == 0)

    # Create a mask where no-star cells are marked as True
    no_star_mask = np.zeros_like(answer, dtype=bool)
    for cell in no_star_cells:
        row, col = cell
        no_star_mask[row, col] = True
    
    return no_star_mask



# specific heuristics functions 

def apply_straight_lines(puzzle, answer):
    no_star_mask = mark_no_star_cells(answer)
    
    # straight line and 3 cell straight line
    made_progress = False
    rows, cols = puzzle.shape

    # Iterate over each region number
    for region in np.unique(puzzle):
        # Find the cells belonging to this region
        region_cells = np.argwhere(puzzle == region)
        
        # Skip processing if any cell in the region is a no-star cell
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        # Check if it's a straight line (same row or same column)
        row_coords, col_coords = zip(*region_cells)
        is_row = len(set(row_coords)) == 1
        is_col = len(set(col_coords)) == 1

        if is_row or is_col:
            if is_row:
                row = row_coords[0]
                sorted_cols = sorted(col_coords)

                # Mark all non-region cells in this row as definite zeros
                for c in range(cols):
                    if puzzle[row, c] != region:
                        answer[row, c] = 0
                        made_progress = True

                # If it's a 3-cell straight line, place stars at both ends
                if len(region_cells) == 3:
                    col_min, col_mid, col_max = sorted_cols
                    answer[row, col_min] = 1
                    answer[row, col_max] = 1
                    answer[row, col_mid] = 0  # Middle cell is a definite zero
                    made_progress = True

            elif is_col:
                col = col_coords[0]
                sorted_rows = sorted(row_coords)

                # Mark all non-region cells in this column as definite zeros
                for r in range(rows):
                    if puzzle[r, col] != region:
                        answer[r, col] = 0
                        made_progress = True

                # If it's a 3-cell straight line, place stars at both ends
                if len(region_cells) == 3:
                    row_min, row_mid, row_max = sorted_rows
                    answer[row_min, col] = 1
                    answer[row_max, col] = 1
                    answer[row_mid, col] = 0  # Middle cell is a definite zero
                    made_progress = True
    if made_progress:
        print(answer)
    zero_surrounding_cells(answer)
    return made_progress, answer
    


def apply_t_shapes(puzzle, answer):
    no_star_mask = mark_no_star_cells(answer)    
#T
    made_progress = False
    rows, cols = puzzle.shape

    # Iterate over each unique region
    for region in np.unique(puzzle):
        # Get all cells belonging to this region
        region_cells = np.argwhere(puzzle == region)
        
        # Skip processing if any cell in the region is a no-star cell
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        # A T-shape must have exactly 4 cells
        if len(region_cells) != 4:
            continue  

        # Convert to a set for easy lookup
        region_set = {tuple(cell) for cell in region_cells}

        # Check all possible T-shape formations
        for r, c in region_cells:
            # Horizontal (Three in a row, one below)
            if {(r, c - 1), (r, c), (r, c + 1), (r + 1, c)}.issubset(region_set):
                # Place stars at both ends of the row, zero the center & bottom
                answer[r, c - 1] = 1
                answer[r, c + 1] = 1
                answer[r, c] = 0
                answer[r + 1, c] = 0
                made_progress = True

            # Horizontal (Three in a row, one above)
            if {(r - 1, c), (r, c - 1), (r, c), (r, c + 1)}.issubset(region_set):
                answer[r, c - 1] = 1
                answer[r, c + 1] = 1
                answer[r, c] = 0
                answer[r - 1, c] = 0
                made_progress = True

            # Vertical (Three in a column, one to the right)
            if {(r - 1, c), (r, c), (r + 1, c), (r, c + 1)}.issubset(region_set):
                answer[r - 1, c] = 1
                answer[r + 1, c] = 1
                answer[r, c] = 0
                answer[r, c + 1] = 0
                made_progress = True

            # Vertical (Three in a column, one to the left)
            if {(r - 1, c), (r, c), (r + 1, c), (r, c - 1)}.issubset(region_set):
                answer[r - 1, c] = 1
                answer[r + 1, c] = 1
                answer[r, c] = 0
                answer[r, c - 1] = 0
                made_progress = True
    if made_progress:
        print(answer)
    zero_surrounding_cells(answer)
    return made_progress, answer


# normal heuristic functions
def apply_l_shapes(puzzle, answer):
    no_star_mask = mark_no_star_cells(answer)

    made_progress = False

    rows, cols = puzzle.shape

    # Iterate over each unique region
    for region in np.unique(puzzle):
        # all cells in the region
        region_cells = np.argwhere(puzzle == region)
        
        # Skip processing if any cell in the region is a no-star cell
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        # An L-shape must have exactly 4 cells
        if len(region_cells) != 4:
            continue  

        # Convert to a set for easy lookup
        region_set = {tuple(cell) for cell in region_cells}

        for r, c in region_cells:

            # Vertical line + right extension
            if {(r - 1, c), (r, c), (r + 1, c), (r + 1, c + 1)}.issubset(region_set):
                answer[r - 1, c] = 1
                answer[r, c] = 0
                # below L as no-star if within bounds
                if r + 2 < rows:
                    answer[r + 2, c] = 0
                    answer[r + 2, c + 1] = 0
                made_progress = True
                made_progress = True

            # Vertical line + left extension
            if {(r - 1, c), (r, c), (r + 1, c), (r + 1, c - 1)}.issubset(region_set):
                answer[r - 1, c] = 1
                answer[r, c] = 0
                if r + 2 < rows:
                    answer[r + 2, c] = 0
                    answer[r + 2, c - 1] = 0
                made_progress = True

            # Vertical line + top right extension
            if {(r - 1, c), (r, c), (r + 1, c), (r - 1, c + 1)}.issubset(region_set):
                answer[r + 1, c] = 1
                answer[r, c] = 0
                # below L as no-star if within bounds
                if r + 2 < rows:
                    answer[r - 2, c] = 0
                    answer[r - 2, c + 1] = 0
                made_progress = True
                made_progress = True

            # Vertical line + left top extension
            if {(r - 1, c), (r, c), (r + 1, c), (r - 1, c - 1)}.issubset(region_set):
                answer[r + 1, c] = 1
                answer[r, c] = 0
                if r + 2 < rows:
                    answer[r - 2, c] = 0
                    answer[r - 2, c - 1] = 0
                made_progress = True

            # Horizontal line + bottom left extension
            if {(r, c - 1), (r, c), (r, c + 1), (r + 1, c - 1)}.issubset(region_set):
                answer[r, c + 1] = 1
                answer[r, c] = 0
                if r + 1 < rows and c - 2 >= 0:
                    answer[r, c - 2] = 0
                    answer[r + 1, c - 2] = 0
                made_progress = True
            
            # Horizontal line + top left extension
            if {(r, c - 1), (r, c), (r, c + 1), (r - 1, c - 1)}.issubset(region_set):
                answer[r, c + 1] = 1
                answer[r, c] = 0
                if r + 1 < rows and c - 2 >= 0:
                    answer[r, c - 2] = 0
                    answer[r - 1, c - 2] = 0
                made_progress = True

            # Horizontal line + bottom right extension
            if {(r, c - 1), (r, c), (r, c + 1), (r + 1, c + 1)}.issubset(region_set):
                answer[r, c - 1] = 1
                answer[r, c] = 0
                if r + 1 < rows and c + 2 < cols:
                    answer[r, c + 2] = 0
                    answer[r + 1, c + 2] = 0
                made_progress = True

            # Horizontal line + top right extension
            if {(r, c - 1), (r, c), (r, c + 1), (r - 1, c + 1)}.issubset(region_set):
                answer[r, c - 1] = 1
                answer[r, c] = 0
                if r + 1 < rows and c + 2 < cols:
                    answer[r, c + 2] = 0
                    answer[r - 1, c + 2] = 0
                made_progress = True

    if made_progress:
        print(answer)
    zero_surrounding_cells(answer)
    return made_progress, answer


def apply_pbdq_shapes(puzzle, answer):
    no_star_mask = mark_no_star_cells(answer)
    made_progress = False
    rows, cols = puzzle.shape

    for region in np.unique(puzzle):
        region_cells = np.argwhere(puzzle == region)

        # Skip processing if any cell in the region is a no-star cell
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        if len(region_cells) != 5:  # Must be exactly 5 cells
            continue

        region_set = {tuple(cell) for cell in region_cells}

        # Try every 2×2 square in the region
        for r, c in region_set:
            if {(r, c), (r + 1, c), (r, c + 1), (r + 1, c + 1)}.issubset(region_set):
                # top left extra square
                if {(r - 1, c)}.issubset(region_set):
                    answer[r-1,c] = 1
                    answer[r,c]= 0
                    answer[r, c+1]= 0
                    made_progress = True
                    if r + 2 < rows:
                        answer[r+2,c]= 0
                        answer[r+2, c+1]= 0
                
                #top right
                if {(r-1,c+1)}.issubset(region_set):
                    answer[r-1,c+1] = 1
                    answer[r,c]= 0
                    answer[r, c+1]= 0
                    made_progress = True
                    if r + 2 < rows:
                        answer[r+2,c]= 0
                        answer[r+2, c+1]= 0
                    
                # bot left
                if {(r +2, c)}.issubset(region_set):
                    answer[r+2,c]= 1
                    answer[r+1, c]= 0
                    answer[r+1, c+1]= 0
                    made_progress = True
                    if r - 1 >= 0 :
                        answer[r-1,c]= 0
                        answer[r-1, c+1]= 0 


                #bot right
                if {(r + 2, c + 1)}.issubset(region_set):
                    answer[r+2,c+1]= 1
                    answer[r+1, c]= 0
                    answer[r+1, c+1]= 0
                    made_progress = True
                    if r - 1 >= 0 :
                        answer[r-1,c]= 0
                        answer[r-1, c+1]= 0 
                    

                #left top
                if {(r, c - 1)}.issubset(region_set):
                    answer[r, c-1]= 1
                    answer[r,c]= 0
                    answer[r+1, c]= 0
                    made_progress= True
                    if c+2 < cols:
                        answer[r, c+2]= 0
                        answer[r+1, c+2]= 0


                #left bottom
                if {(r + 1, c - 1)}.issubset(region_set):
                    answer[r+1 , c-1]= 1
                    answer[r,c]= 0
                    answer[r+1, c]= 0
                    made_progress= True
                    if c+2 < cols:
                        answer[r, c+2]= 0
                        answer[r+1, c+2]= 0

                # right top
                if {(r, c + 2)}.issubset(region_set):
                    answer[r,c+2]= 1
                    answer[r, c+1]= 0
                    answer[r+1, c+1]= 0
                    made_progress = True
                    if c-1 >=0:
                        answer[r, c-1]= 0
                        answer[r+1, c-1]= 0 

                #right bottom
                if {(r + 1, c + 2)}.issubset(region_set):
                    answer[r+1,c+2]= 1
                    answer[r, c+1]= 0
                    answer[r+1, c+1]= 0
                    made_progress = True
                    if c-1 >=0:
                        answer[r, c-1]= 0
                        answer[r+1, c-1]= 0

    if made_progress:
        print(answer) 

    zero_surrounding_cells(answer)
    return made_progress, answer




def apply_offset_stack(puzzle, answer):
    no_star_mask = mark_no_star_cells(answer)
    
    made_progress = False
    
    # Iterate over each unique region
    for region in np.unique(puzzle):
        # Get all cells in the region
        region_cells = np.argwhere(puzzle == region)
        region_cells = [cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]]

        

        # The shape must have exactly 4 cells
        if len(region_cells) != 4:
            continue

        # Convert to a set for easy lookup
        region_set = {tuple(cell) for cell in region_cells}

        for r, c in region_cells:
            # Check for a Z-shape (horizontal)
            if {(r, c), (r, c + 1), (r + 1, c - 1), (r + 1, c)}.issubset(region_set):
                answer[r, c+1] = 1
                answer[r + 1, c - 1] = 1
                made_progress = True

            # Check for an S-shape (horizontal)
            if {(r, c), (r, c + 1), (r + 1, c + 1), (r + 1, c + 2)}.issubset(region_set):
                answer[r, c] = 1
                answer[r + 1, c + 2] = 1
                made_progress = True

            # Check for a Z-shape (vertical)
            if {(r, c), (r + 1, c), (r + 1, c + 1), (r + 2, c + 1)}.issubset(region_set):
                answer[r, c] = 1
                answer[r + 2, c + 1] = 1
                made_progress = True

            # Check for an S-shape (vertical)
            if {(r, c), (r + 1, c), (r + 1, c - 1), (r + 2, c - 1)}.issubset(region_set):
                answer[r, c] = 1
                answer[r + 2, c - 1] = 1
                made_progress = True


    if made_progress:
        print(answer)
    zero_surrounding_cells(answer)
    return made_progress, answer



def apply_6_cell_rectangle(puzzle, answer):
    made_progress = False
    no_star_mask = mark_no_star_cells(answer)

    answer = np.array(answer, dtype=object)
    rows, cols = puzzle.shape

    for region in np.unique(puzzle):
        region_cells = np.argwhere(puzzle == region)
        region_cells = np.array([cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]])

        if len(region_cells) != 6:
            continue

        min_r, max_r = region_cells[:, 0].min(), region_cells[:, 0].max()
        min_c, max_c = region_cells[:, 1].min(), region_cells[:, 1].max()

        height = max_r - min_r + 1
        width = max_c - min_c + 1

        # Identify 3×2 (tall) and 2×3 (wide) shapes
        if {height, width} == {3, 2}:  # Tall 3×2 shape
            center_row = min_r + 1  # Middle row
            center_cells = [(center_row, min_c), (center_row, min_c + 1)]

        elif {height, width} == {2, 3}:  # Wide 2×3 shape
            center_col = min_c + 1  # Middle column
            center_cells = [(min_r, center_col), (min_r + 1, center_col)]

        else:
            continue  # Ignore non-6-cell regions

        # Deduct the middle row or column
        for r, c in center_cells:
            if 0 <= r < rows and 0 <= c < cols and answer[r, c] is None:
                answer[r, c] = 0
                made_progress = True

    if made_progress:
        print(answer)

    return made_progress, answer





def apply_9_cell_square(puzzle, answer):
    made_progress = False
    no_star_mask = mark_no_star_cells(answer)

    answer = np.array(answer, dtype=object)  

    for region in np.unique(puzzle):
        region_cells = np.argwhere(puzzle == region)
        region_cells = np.array([cell for cell in region_cells if not no_star_mask[cell[0], cell[1]]])

        if len(region_cells) != 9:
            continue

        min_r, max_r = region_cells[:, 0].min(), region_cells[:, 0].max()
        min_c, max_c = region_cells[:, 1].min(), region_cells[:, 1].max()

        height = max_r - min_r + 1
        width = max_c - min_c + 1

        if height == 3 and width == 3:  
            center_cell = (min_r + 1, min_c + 1)  

            if answer[center_cell] is None:
                answer[center_cell] = 0
                made_progress = True

    if made_progress:
        print(answer)
    
    return made_progress, answer




def apply_star_surrounds(puzzle, answer):
    made_progress = False
    answer = np.array(answer, dtype=object)  # Ensure answer is a NumPy array

    rows, cols = puzzle.shape

    # Check rows
    for r in range(rows):
        if np.sum(np.array(answer[r]) == 1) == 2:  # If row has enough stars
            for c in range(cols):
                if answer[r, c] is None:  # Set remaining cells in the row to 0
                    answer[r, c] = 0
                    made_progress = True

    # Check columns
    for c in range(cols):
        if np.sum(np.array(answer[:, c]) == 1) == 2:  # If column has enough stars
            for r in range(rows):
                if answer[r, c] is None:  # Set remaining cells in the column to 0
                    answer[r, c] = 0
                    made_progress = True
    
    if made_progress:
        print(answer)

    return made_progress, answer



# hard heuristic functions

def apply_rule_of_crowding(puzzle, answer):
    # Implement Rule of Crowding logic
    return False  

def apply_rule_of_four_squares(puzzle, answer):
    # Implement Rule of Four-Squares logic
    return False  

def apply_rule_of_clumps(puzzle, answer):
    # Implement Rule of Clumps logic
    return False  



In [7]:
def classify_difficulty(puzzle, solution):
    #Classifies a Star Battle puzzle into Normal, Hard, or Extreme
    answer = np.empty_like(puzzles[0], dtype=object)
    answer[:] = None    
    potential_stars = np.zeros_like(puzzle)


    if normal_solver(puzzle, solution, answer):
        return "Normal"
    # elif hard_solver(puzzle, solution, answer):
    #     return "Hard"
    else:
        return "Extreme"

def normal_solver(puzzle, solution, answer):
    #attempts to solve using normal heuristics
    for _ in range(100):  # Limit iterations to simulate human solving
        made_progress = False

        # Call heuristic functions and capture both progress and updated answer
        progress, answer = apply_straight_lines(puzzle, answer)
        made_progress |= progress  # Keep track if any progress was made
        
        progress, answer = apply_t_shapes(puzzle, answer)
        made_progress |= progress  

        progress, answer = apply_l_shapes(puzzle, answer)
        made_progress |= progress  

        progress, answer = apply_pbdq_shapes(puzzle, answer)
        made_progress |= progress  

        progress, answer = apply_offset_stack(puzzle, answer)
        made_progress |= progress  

        progress, answer = apply_6_cell_rectangle(puzzle, answer)
        made_progress |= progress  

        progress, answer = apply_9_cell_square(puzzle, answer)
        made_progress |= progress  

        progress, answer = apply_star_surrounds(puzzle, answer)
        made_progress |= progress  



        # If the answer is solved, return early
        answer_copy = np.copy(answer)  
        answer_copy[answer_copy != 1] = 0
        if np.array_equal(answer_copy, solution):
            return True

        # No progress made, break out and try the hard solver
        if not made_progress:
            break  # No more easy deductions, try hard

    answer_copy = np.copy(answer)  
    answer_copy[answer_copy != 1] = 0  # Set any value not equal to 1 to 0
    
    return np.array_equal(answer_copy, solution)  # Check if puzzle is fully solved

def hard_solver(puzzle, solution, answer):
    #attempts solve using normal and hard heuristics
    for _ in range(10):  # limited iterations
        made_progress = False

        made_progress |= apply_rule_of_crowding(puzzle, answer)
        made_progress |= apply_rule_of_four_squares(puzzle, answer)
        made_progress |= apply_rule_of_clumps(puzzle, answer)
        made_progress |= normal_solver(puzzle, answer)

        if not made_progress:
            break  
        
        
    answer_copy = np.copy(answer)  
    answer_copy[answer_copy != 1] = 0  # Set any value not equal to 1 to 0
    
    return np.array_equal(answer_copy, solution)  # Check if puzzle is fully solved

# testing for the functions


In [8]:
answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
answer[:] = None
answer_copy = np.copy(answer)  # Create a copy of the answer array
answer_copy[answer_copy != 1] = 0  # Set any value not equal to 1 to 0


print(answer)
print(answer_copy)

[[None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]]
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [9]:
answer2 = np.array([
    [-1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1]
])

print(answer2)

answer_copy2 = zero_surrounding_cells(answer2.copy())  # Call the function on a copy
print(answer_copy2)


[[-1 -1 -1 -1 -1]
 [-1 -1  1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]
[[-1  0  0  0 -1]
 [-1  0  1  0 -1]
 [-1  0  0  0 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]


In [10]:
puzzletry = puzzles[1].copy()
print(puzzletry)
solutiontry = solutions[1].copy()
print(solutiontry)
egpuzzle = mark_no_star_cells(puzzletry)
print(egpuzzle)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None    
    print(apply_straight_lines(puzzles[i], answer)[0])
    
print(puzzles[0])
print(puzzles[1])

False
True
True
False
True
False
True
True
False
True
True
False
True
False
True
True
False
True
False
False
[[9 1 1 1 1 7 7 7 7 7]
 [9 9 1 1 1 7 7 7 7 7]
 [9 9 1 1 1 1 6 7 0 0]
 [9 1 1 1 1 1 6 0 0 0]
 [9 2 1 1 1 6 6 0 0 0]
 [2 2 8 8 3 3 6 0 0 0]
 [2 2 8 8 3 3 3 4 0 0]
 [2 2 8 4 4 4 4 4 4 4]
 [2 2 8 5 5 5 5 4 4 4]
 [2 2 5 5 5 5 5 4 4 4]]
[[1 1 1 1 6 6 6 6 6 6]
 [1 1 1 7 6 6 6 6 6 5]
 [1 1 1 7 0 0 0 0 5 5]
 [1 1 1 7 3 0 4 5 5 5]
 [1 1 1 3 3 3 4 5 5 5]
 [1 3 3 3 3 3 4 4 5 5]
 [3 3 3 3 2 2 4 4 4 4]
 [3 3 3 3 2 2 4 4 9 9]
 [8 8 8 3 2 2 2 9 9 9]
 [8 8 8 8 2 2 2 2 9 9]]


In [ ]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    
    
    print(apply_l_shapes(puzzles[i], answer)[0])
    
print(puzzles[19])
print(apply_l_shapes(puzzles[19], answer)[1])


False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
False
True
False
True
[[4 4 4 6 2 2 2 2 7 7]
 [4 4 6 6 6 2 2 3 7 7]
 [4 4 6 6 6 6 2 3 3 7]
 [4 6 6 6 6 6 3 3 3 7]
 [1 1 1 6 6 3 3 3 9 8]
 [1 1 1 6 6 3 3 3 9 8]
 [0 1 1 6 5 5 3 9 9 8]
 [0 0 0 0 5 5 5 8 8 8]
 [0 0 0 5 5 5 5 8 8 8]
 [0 0 0 5 5 5 5 5 8 8]]
[[None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None 0 0 0]
 [None None None None None None None 0 1 0]
 [None None None None None None None 0 0 0]
 [None None None None None None None None None None]
 [None None None None None None None 0 0 None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]]


In [ ]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
    # Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_t_shapes(puzzles[i], answer)[0])
    
print(puzzles[18])
print(puzzles[19])

False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
[[3 3 3 3 8 8 8 8 8 8]
 [3 3 3 3 8 8 8 8 8 8]
 [3 3 3 7 7 8 8 8 8 0]
 [3 3 3 7 7 1 0 0 0 0]
 [3 9 7 7 7 1 0 0 0 0]
 [9 9 9 9 1 1 1 4 4 4]
 [9 9 9 9 1 1 2 2 2 4]
 [5 9 9 6 6 6 2 2 2 4]
 [5 5 6 6 6 6 6 2 2 2]
 [5 5 6 6 6 6 2 2 2 2]]
[[4 4 4 6 2 2 2 2 7 7]
 [4 4 6 6 6 2 2 3 7 7]
 [4 4 6 6 6 6 2 3 3 7]
 [4 6 6 6 6 6 3 3 3 7]
 [1 1 1 6 6 3 3 3 9 8]
 [1 1 1 6 6 3 3 3 9 8]
 [0 1 1 6 5 5 3 9 9 8]
 [0 0 0 0 5 5 5 8 8 8]
 [0 0 0 5 5 5 5 8 8 8]
 [0 0 0 5 5 5 5 5 8 8]]


In [11]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_pbdq_shapes(puzzles[i], answer)[0])
    
print(puzzles[18])
print(puzzles[19])


False


IndexError: index 18 is out of bounds for axis 0 with size 1

In [12]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_offset_stack(puzzles[i], answer)[0])

False


In [13]:
#OFSET STACK FUCNTION CHANGES ITS MIND
puzzle = np.array([
        [4, 4, 4, 6, 2, 2, 2, 2, 7, 7],
        [4, 4, 6, 6, 6, 2, 2, 3, 7, 7],
        [4, 4, 6, 6, 6, 6, 2, 3, 3, 7],
        [4, 6, 6, 6, 6, 6, 3, 3, 3, 7],
        [1, 1, 1, 6, 6, 3, 3, 3, 9, 8],
        [1, 1, 1, 6, 6, 3, 3, 9, 9, 8], 
        [0, 1, 1, 6, 5, 5, 3, 9, 8, 8], 
        [0, 0, 0, 0, 5, 5, 5, 8, 8, 8],
        [0, 0, 0, 5, 5, 5, 5, 8, 8, 8],
        [0, 0, 0, 5, 5, 5, 5, 5, 8, 8]
    ])

# Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
answer[:] = None
print(apply_offset_stack(puzzle, answer)[0])

[[None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None 1 None]
 [None None None None None None None None None None]
 [None None None None None None None 1 None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]]
True


In [47]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_6_cell_rectangle(puzzles[i], answer)[0])
    
print(puzzles[16])
print(apply_6_cell_rectangle(puzzles[16], answer)[1])
print(puzzles[17])

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
[[7 7 6 2 2 2 8 8 8 8]
 [7 7 6 2 2 2 0 8 8 8]
 [7 7 6 2 2 0 0 8 8 8]
 [6 6 6 6 6 0 0 0 8 8]
 [3 3 3 6 0 0 0 0 8 8]
 [3 3 3 3 0 0 0 0 8 8]
 [4 4 3 3 1 9 9 9 9 9]
 [4 4 1 1 1 1 9 9 9 9]
 [4 4 4 1 1 5 5 5 9 9]
 [4 4 4 1 1 5 5 5 5 5]]
[[None None None None None None None None None None]
 [0 0 None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]]
[[9 9 9 9 8 8 8 8 8 2]
 [9 9 9 9 0 1 1 4 4 2]
 [9 7 9 9 0 1 1 4 4 2]
 [7 7 7 0 0 1 4 4 4 2]
 [7 7 7 7 1 1 1 4 4 2]
 [7 7 7 7 

In [29]:
puzzle = np.array([
        [4, 4, 4, 6, 2, 2, 2, 2, 7, 7],
        [4, 4, 6, 6, 6, 2, 2, 3, 7, 7],
        [4, 4, 6, 6, 6, 6, 2, 3, 3, 7],
        [4, 6, 6, 6, 6, 6, 3, 3, 3, 7],
        [1, 1, 1, 6, 6, 3, 3, 3, 9, 8],
        [1, 1, 1, 6, 6, 3, 3, 9, 9, 8], 
        [0, 1, 1, 6, 5, 5, 3, 9, 8, 8], 
        [0, 0, 1, 1, 5, 5, 5, 8, 8, 8],
        [0, 0, 1, 5, 5, 5, 5, 8, 8, 8],
        [1, 1, 1, 5, 5, 5, 5, 5, 8, 8]
    ])

# Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
answer[:] = None
print(apply_pbdq_shapes(puzzle, answer)[0])

True


In [36]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_6_cell_rectangle(puzzles[i], answer)[0])
    
print(puzzles[16])
print(apply_6_cell_rectangle(puzzles[16], answer)[1])
print(puzzles[17])



False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
[[7 7 6 2 2 2 8 8 8 8]
 [7 7 6 2 2 2 0 8 8 8]
 [7 7 6 2 2 0 0 8 8 8]
 [6 6 6 6 6 0 0 0 8 8]
 [3 3 3 6 0 0 0 0 8 8]
 [3 3 3 3 0 0 0 0 8 8]
 [4 4 3 3 1 9 9 9 9 9]
 [4 4 1 1 1 1 9 9 9 9]
 [4 4 4 1 1 5 5 5 9 9]
 [4 4 4 1 1 5 5 5 5 5]]
[[0 0 None None None None None None None None]
 [0 0 None None None None None None None None]
 [0 0 None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]]
[[9 9 9 9 8 8 8 8 8 2]
 [9 9 9 9 0 1 1 4 4 2]
 [9 7 9 9 0 1 1 4 4 2]
 [7 7 7 0 0 1 4 4 4 2]
 [7 7 7 7 1 1 1 4 4 2]
 [7 7 7 7 1 1 1 4 4 2]

In [39]:
for i in range(len(puzzles)): 

    puzzle = puzzles[i]
    solution = solutions[i]

    # Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
    answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
    answer[:] = None
    print(apply_9_cell_square(puzzles[i], answer)[0])

print(puzzles[9])   
print(apply_9_cell_square(puzzles[9], answer)[1])


False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
[[3 3 7 7 7 2 2 2 6 6]
 [3 3 7 7 7 2 2 2 6 6]
 [3 3 7 7 7 2 2 6 6 6]
 [9 3 3 4 4 4 2 6 6 6]
 [9 3 3 4 4 4 4 6 6 6]
 [9 3 3 4 4 4 0 0 0 0]
 [9 3 3 1 4 4 8 5 5 5]
 [9 3 1 1 1 8 8 8 5 5]
 [9 3 1 1 1 8 8 8 8 5]
 [9 1 1 1 8 8 8 8 8 5]]
[[None None None None None None None None None None]
 [None None None 0 None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]]


In [50]:
puzzle = np.array([
        [4, 4, 4, 6, 2, 2, 2, 2, 7, 7],
        [4, 4, 6, 6, 6, 2, 2, 3, 7, 7],
        [4, 4, 6, 6, 6, 6, 2, 3, 3, 7],
        [4, 6, 6, 6, 6, 6, 3, 3, 3, 7],
        [1, 1, 1, 6, 6, 3, 3, 9, 9, 8],
        [1, 1, 1, 6, 6, 3, 3, 9, 9, 8], 
        [0, 1, 1, 6, 5, 5, 3, 9, 9, 8], 
        [0, 0, 1, 1, 5, 5, 5, 8, 8, 8],
        [0, 0, 1, 5, 5, 5, 5, 8, 8, 8],
        [1, 1, 1, 5, 5, 5, 5, 5, 8, 8]
    ])

# Reset tracking arrays for each puzzle
# Reset tracking arrays for each puzzle
answer = np.empty_like(puzzles[0], dtype=object)  # Use dtype=object for None or custom placeholders
answer[:] = None
print(apply_6_cell_rectangle(puzzle, answer)[0])

True


# classify

In [114]:
# for i in range(len(puzzles)): 

#     puzzle = puzzles[i]
#     solution = solutions[i]

#     print(classify_difficulty(puzzles[i], answer))


print(classify_difficulty(puzzles[0], answer))

[[None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None 0 None 0 None None None None]
 [None None None 0 None 0 1 None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]
 [None None None None None None None None None None]]
[[None None 0 None None None None None None None]
 [None None 0 None None None None None None None]
 [None None 0 None None None None None None None]
 [None None 0 None None None None None None None]
 [None None 0 None None None None None None None]
 [None None None 0 None 0 0 0 None None]
 [None None None 0 None 0 1 0 None None]
 [None None None None None 0 0 0 None None]
 [None None None None None None None None None None]
 [None None 0 None None None None None None None]]
[[No

In [ ]:
# Classify each puzzle
difficulty_labels = [classify_difficulty(puzzles[i], solutions[i]) for i in range(len(puzzles))]

# Save labeled data
np.savez_compressed("star_battle_labeled_dataset.npz", puzzles=puzzles, solutions=solutions, labels=np.array(difficulty_labels))
print("Labeled dataset saved successfully!")

In [ ]:
# dataset = np.load("star_battle_dataset.npz")
# puzzles = dataset["puzzles"]
# solutions = dataset["solutions"]

difficulties = []
for i in range(len(puzzles)):
    difficulty_score = classify_difficulty(puzzles[i], solutions[i])
    
    

# Save difficulty labels
print(difficulties)
# np.savez_compressed("star_battle_dataset_with_difficulty.npz", puzzles=puzzles, solutions=solutions, difficulties=difficulties)
# print("Dataset with difficulty scores saved!")


['Hard', 'Hard', 'Hard', 'Hard', 'Hard']
Dataset with difficulty scores saved!
